In [1]:
#Imports
import ccxt
import os
# import pandas as pd
import requests
from Historic_Crypto import HistoricalData
from Historic_Crypto import LiveCryptoData
from Historic_Crypto import Cryptocurrencies
from pytrends.request import TrendReq
from dotenv import load_dotenv
from tqdm import tqdm_notebook as tqdm

In [2]:
load_dotenv()
#Telegram Bot and channel information
bot_token = os.getenv("bot_api_token")
telegram_channel_id = os.getenv("telegram_channel_id")

# Preparing ccxt engine
exchange_id = 'coinbasepro'

exchange_class = getattr(ccxt, exchange_id)

exchange = exchange_class({'timeout': 30000,
                           'enableRateLimit': True})


In [3]:
#Google Trends - Interest over time
def get_trend_data(kw):
    try:
        pytrends = TrendReq(hl='en-US', 
                            tz=600)
        
        pytrends.build_payload([kw], 
                               cat=0, 
                               timeframe='now 1-H', 
                               geo='', 
                               gprop='')
        
        df = pytrends.interest_over_time()
        
        return df.iloc[-1][0]
    except:
        return 0

# Function is to get OHLCV data for a ticker over 1 day period. It uses ccxt library to get the data.
# product - Crypto asset to get the data for. Eg. BTC/USD
# time - Current time in epoch
def retrieve_ohlcv(product):
    
    return exchange.fetch_ohlcv(product, '1m')

def get_current_price(product):
    price = retrieve_ohlcv(product)[-1][4]
    return price

# Function is to get historical data for a ticker as far back as the start date provided at the interval of 1 day. It uses Historic_Crypto library to get the data.
# product - Crypto asset to get the data for. Eg. BTC-USD
# start_date -  Start date as a string in the format YYYY-MM-DD-HH-MM (str)
def retrieve_historical_data(product,start_date):
    df = HistoricalData(product, 86400, start_date).retrieve_data()
    return df

#Sending message to Telegram channel
def telegram_bot_sendtext(bot_message):
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + telegram_channel_id + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)

    return response.json()